# Setting Land Cover for Shetran

This notebook was created to demostrate the process of creating the Land Cover file use in SHETRAN based in the catchment extent

---

#### Author: 
                LF Velasquez & I Rohrmueller - Newcastle University

#### Date:
                Nov 2022

#### Version:
                1.0

#### Notes:
                - To get jupyter env version type `!jupyter --version` in a python cell
            
#### Jupyter version:

#### Python version:

---

# Notebook set-up

## 1. Setting Python Modules

In [1]:
import geopandas as gpd
import pandas as pd
from pathlib import Path
import rasterstats
import rioxarray

# ODC modules
import datacube

# Custom modules
import utils

# Modules for config.ini
from configparser import ConfigParser
config = ConfigParser()

## 2. Global variables

In [6]:
# Setting the path to the work environment
dir_abs = Path().resolve().parent.parent
dir_abs

# Read config file values
config.read('config.ini')

# Setting CRS
crs_global = config.getint('crs_setting', 'GLB')
crs_local = config.getint('crs_setting', 'COL')

# Open Data Cube Product
dc_data = config.get('dc_product', 'LC')

# No data value
ND = config.getint('res_setting', 'NO_DATA')

## 3. Start of Process

### Reading shapefile

In [3]:
# Read shp file to geopandas dataframe
grid_path = Path(dir_abs / 'shetran_data/active_data/final_mask_wgs84.shp')
grid = gpd.read_file(grid_path)

xmin, ymin, xmax, ymax = grid.total_bounds
grid

,intersect,SHETRAN_ID,geometry
0,0,-9999,"POLYGON ((-76.70463 3.59102, -76.68665 3.59107..."
1,0,-9999,"POLYGON ((-76.70458 3.57295, -76.68660 3.57300..."
2,0,-9999,"POLYGON ((-76.70453 3.55488, -76.68655 3.55493..."
3,0,-9999,"POLYGON ((-76.70448 3.53681, -76.68650 3.53687..."
4,0,-9999,"POLYGON ((-76.70443 3.51875, -76.68644 3.51880..."
...,...,...,...
793,1,0,"POLYGON ((-76.03853 3.30348, -76.02054 3.30351..."
794,0,-9999,"POLYGON ((-76.03849 3.28540, -76.02051 3.28544..."
795,0,-9999,"POLYGON ((-76.03846 3.26733, -76.02047 3.26736..."
796,0,-9999,"POLYGON ((-76.03842 3.24925, -76.02044 3.24929..."


### Reading Land Cover data from ODC

In [4]:
# Set ODC application
dc = datacube.Datacube(app="worldcover")

# # Set dask client
# client = dask.distributed.Client()
# display(client)

# Get data
# Load data from the datacube
buffer = 0.125
ds = dc.load(product=dc_data,
             lat=(ymin - buffer, ymax + buffer),
             lon=(xmin - buffer, xmax + buffer),
             time=('2021'),
            #  dask_chunks={'time': 1, 'longitude': 200, 'latitude': 200}
             )

# Print output data
ds

<xarray.Dataset>
Dimensions:         (time: 1, latitude: 7574, longitude: 11212)
Coordinates:
  * time            (time) datetime64[ns] 2021-07-02
  * latitude        (latitude) float64 3.718 3.718 3.718 ... 3.087 3.087 3.087
  * longitude       (longitude) float64 -76.83 -76.83 -76.83 ... -75.9 -75.9
    spatial_ref     int32 4326
Data variables:
    classification  (time, latitude, longitude) uint8 10 10 10 10 ... 10 10 10
Attributes:
    crs:           EPSG:4326
    grid_mapping:  spatial_ref

### Create temp .tif file
This uses the information read from ODC

In [5]:
_tif = Path(dir_abs / 'shetran_data/temp_data/ea_worldcover.tif')
ds['classification'].rio.to_raster(_tif)


### Perform zonal stats for the catcment

In [8]:
# Find the mean elevation for each grid in the catchment
# This produces a dict following the same order as the grid geodataframe
catchm_stats_lc = rasterstats.zonal_stats(str(grid_path), str(_tif), categorical=True, nodata=ND)

# Create a list containing the dictionary key (land cover type) with the Max value for each grid in the catchment
lc_largest = [max(lc_grid, key=lc_grid.get) for lc_grid in catchm_stats_lc]

# Add the largest lc to grid geodataframe
grid['lc_largest'] = lc_largest

# Replacing 0 with -9999
grid.loc[grid['lc_largest'] == 0, 'lc_largest'] = ND

grid

,intersect,SHETRAN_ID,geometry,lc_largest
0,0,-9999,"POLYGON ((-76.70463 3.59102, -76.68665 3.59107...",10
1,0,-9999,"POLYGON ((-76.70458 3.57295, -76.68660 3.57300...",30
2,0,-9999,"POLYGON ((-76.70453 3.55488, -76.68655 3.55493...",10
3,0,-9999,"POLYGON ((-76.70448 3.53681, -76.68650 3.53687...",10
4,0,-9999,"POLYGON ((-76.70443 3.51875, -76.68644 3.51880...",10
...,...,...,...,...
793,1,0,"POLYGON ((-76.03853 3.30348, -76.02054 3.30351...",30
794,0,-9999,"POLYGON ((-76.03849 3.28540, -76.02051 3.28544...",30
795,0,-9999,"POLYGON ((-76.03846 3.26733, -76.02047 3.26736...",30
796,0,-9999,"POLYGON ((-76.03842 3.24925, -76.02044 3.24929...",30


### Delete temp .tif file
This avoid over-bloating in case someone forgets to delete the file

In [9]:
utils.file_remove(_tif)

ea_worldcover.tif DELETED!


### Working with the land cover geodataframe to set the final output

In [10]:
# Get the centroid for each grid
grid_bg = grid.to_crs(crs_local)
grid_bg['centroid'] = grid_bg['geometry'].centroid

# Create lat (Y) and lon (X) columns
grid_bg['lat'] = grid_bg['centroid'].y.astype(int)
grid_bg['lon'] = grid_bg['centroid'].x.astype(int)

# Change geopandas to pandas ready to create csv file for the mean and min elevation
df_grid_lc = pd.DataFrame(grid_bg[['lc_largest', 'lat', 'lon']].copy())

# Pivoting dataframe to replicate SHETRAN format
# Pivoting dataframe using lon as column and lat as row
df_pivot_lc = df_grid_lc.pivot(index='lat', columns='lon', values='lc_largest')

# Remove any NaN
df_pivot_lc = df_pivot_lc.fillna(0)

# with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#     print(df_pivot_lc)
display(df_pivot_lc)

lon,709018,711018,713018,715018,717018,719018,721018,723018,725018,727018,...,765018,767018,769018,771018,773018,775018,777018,779018,781018,783018
lat,,,,,,,,,,,,,,,,,,,,,
848266,10,10,10,10,10,10,10,10,40,40,...,10,10,10,10,10,10,30,30,30,10
850266,10,10,10,10,30,10,10,30,30,40,...,10,10,10,10,10,30,30,30,30,30
852266,10,10,10,10,10,10,10,30,50,50,...,10,10,10,10,10,30,30,30,30,30
854266,10,10,10,10,10,10,10,10,30,30,...,10,10,10,10,10,10,10,30,30,30
856266,10,10,10,10,10,10,10,10,30,30,...,30,10,10,10,10,10,30,30,30,30
858266,10,10,10,10,10,10,10,10,10,30,...,10,10,10,10,10,10,30,10,30,30
860266,10,10,10,10,10,10,10,10,10,10,...,10,10,10,10,10,10,30,30,30,30
862266,10,10,10,10,10,10,10,10,10,10,...,10,10,10,10,10,10,30,30,30,30
864266,10,10,10,10,10,10,10,10,10,50,...,10,10,10,10,10,10,30,30,30,30


### Saving the final outpt as csv file

In [12]:
# Creating the text file for min elevation
utils.shetran_csv_file(dir_abs, 'final_land_cover_SHETRAN', df_pivot_lc, 'd')
print('SHETRAN Land Cover file created')


SHETRAN Land Cover file created
